

# Les Import :

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece


     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=13b43d82ce1764cc3da1b54e0ac2d6034703081b60c91c6f1c10c317ec0531a1
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=ead8740852569f3dfff6b5c770d77a396f811fc457014b12319262955264027c
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=f3fde3db8eb00863ec4a4930847cc4b74db12023925585cefdae712b889a7387
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 17.3MB/s 


In [ ]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from bs4 import BeautifulSoup #to extract words from HTML documents

import string
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam,Adagrad,Adadelta,Adamax
from keras.models import Sequential

from random import shuffle
def shuffle_list(*ls):
  l =list(zip(*ls))
  shuffle(l)
  return zip(*l)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# DL des datas :

Cette section sert a download les datas. Il faut donc la modifier selon la facon dont on utilise ce fichier.

In [ ]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



"train.zip" :

In [ ]:
downloaded = drive.CreateFile({'id':"1l6OX6DbY00CE392f-cYZtOZ0lkM"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train.tsv.zip')

"Tweets-Flight.csv" :

In [ ]:
downloaded = drive.CreateFile({'id':"1V96pme-eFHBOerhM79tNhRtwoh2VNkDD"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Tweets-Flight.csv')

On va load le fichier des datas. Exexuter la cellule qui correspond au bon fichier. NE PAS executer les 2 cellules.

# **MovieReviews :**

In [ ]:
movie_reviews = pd.read_csv("train.tsv.zip", sep="\t")
rev0=movie_reviews['Phrase']
sentiment0=movie_reviews.Sentiment.values
#y=to_categorical(movie_reviews.Sentiment.values)
print("Nombre total de Reviews :"+ str(len(sentiment0)))

rev,sentiment=shuffle_list(rev0,sentiment0)

# On re-equilible la base de données : 

n=120000

k1=0
k2=0
k3=0
y0=[]
reviews0=[]
for i in range(len(sentiment)):
  el=sentiment[i]
  if el<=1 and k1<n/3 : 
    k1+=1
    y0.append([1,0,0])
    reviews0.append(rev[i])
  if el==2 and k2<n/3:
    k2+=1
    y0.append([0,1,0])
    reviews0.append(rev[i])
  if el>=3 and k3<n/3 :
    k3+=1
    y0.append([0,0,1])
    reviews0.append(rev[i])


y=np.array(y0,dtype='float32')
reviews=np.array(reviews0)
print("Nombre total de Reviews apres equilibrage :"+ str(len(y)))

Nombre total de Reviews :156060
Nombre total de Reviews apres equilibrage :114345


# **OU**

# **Tweets-flight :**

In [ ]:
tweet_reviews = pd.read_csv('Tweets-Flight.csv')
rev=np.array(tweet_reviews['text'])
sentiment=tweet_reviews['airline_sentiment']
y0=[]
reviews0=[]

# On re-equilible la base de données : on doit remove 5500 'neutral'
n=9178-5500
k=0

for i in range(len(sentiment)):
  el=sentiment[i]
  if el=='negative' and k<n:
    k+=1 
    y0.append([1,0,0])
    reviews0.append(rev[i])

  if el=='neutral': 
    y0.append([0,1,0])
    reviews0.append(rev[i])

  if el=='positive': 
    y0.append([0,0,1])
    reviews0.append(rev[i])

y=np.array(y0,dtype='float32')
reviews=np.array(reviews0)

print("Nombre total de Tweets :"+ str(len(reviews)))

Nombre total de Tweets :9140


Fin du load

Visualisation des datas :

In [ ]:
print(reviews)
print(y)

['@VirginAmerica What @dhepburn said.'
 "@VirginAmerica plus you've added commercials to the experience... tacky."
 "@VirginAmerica I didn't today... Must mean I need to take another trip!"
 ... '@AmericanAir thank you we got on a different flight to Chicago.'
 '@AmericanAir Please bring American Airlines to #BlackBerry10'
 '@AmericanAir we have 8 ppl so we need 2 know how many seats are on the next flight. Plz put us on standby for 4 people on the next flight?']
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


# **Shuffle Des données :**





In [ ]:
y,reviews= shuffle_list(y,reviews)
y=np.array(y)
reviews=np.array(reviews)

# Remove Stop-Words :

In [ ]:
import nltk
import re
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
reviews=np.array([text_preprocessing(s)for s in reviews])

# **Tokenizer :**

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

# **Padding :**

In [ ]:
def PaddingX(M,pad=-1):
  if pad==-1: 
    pad = len(max(M, key=len))
  return np.array([i + [0]*(pad-len(i)) for i in M]),pad

In [ ]:
#padded_reviews,padd=PaddingX(tokenized_reviews,padd)
padded_reviews,padd=PaddingX(tokenized_reviews)

print(padd)

40


# **Separation train/validation :**

In [ ]:
# Create X and Y
y_target=y
num_classes = len(y_target[0])

X_train,X_val,y_train,y_val = train_test_split(padded_reviews, y_target, test_size=0.2, stratify=y_target)

# **Définition du réseau Convolution :**

In [ ]:
def Model_CV_PL_FL(with_early_stop=False):
  D = 50 #embedding dimensionality
  dropout_rate = 0.5

  # Define early stopping that will be used as callback
  early_stopping = EarlyStopping(min_delta=0.01, mode='max', monitor='val_accuracy', patience=3) # min_delta par defaut = 0.01


  if with_early_stop : callback=[]
  else :callback = [early_stopping]


  sequence_length=X_train.shape[1]


  # add layers to model
  model = Sequential()
  model.add(Embedding(len(tokenizer.vocab), D))
  model.add(Reshape((sequence_length,D,1)))
  model.add(Conv2D(40, (3, D), activation='relu' ))
  model.add(GlobalMaxPooling2D())
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(num_classes,activation='softmax'))

  # use a relatively low learning rate
  model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

  model.summary()

  r = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=256, callbacks=callback)
  #plt.plot(r.history['loss'][5:])
  return model

# **Apprentissage des Modeles :**

In [ ]:
model_trained=Model_CV_PL_FL(with_early_stop=True)

# **Détails des resultats :**

In [ ]:
Y_sortie_proba=model_trained.predict(X_val)
Y_sortie=Y_sortie_proba.argmax(1)
Y_val=y_val.argmax(1)

Matrice=np.zeros((3,3))
for i in range(len(Y_sortie)):
  Matrice[Y_sortie[i]][Y_val[i]]+=1

reussite=[0 for i in range(num_classes)]
for i in range(len(y_val)):
  reussite[Y_val[i]]+=(Y_sortie[i]==Y_val[i])

print("Nombre de reussites : " + str(sum(reussite))+" / "+str(len(Y_val)) )
taux_reussite=[reussite[i]/sum(Y_val==i) for i in range(num_classes)]
print("Pourcentage de reussites par sentiment : " + str(taux_reussite))
print("Matrice des resultats : ")
print(Matrice)

Nombre de reussites : 1348 / 1828
Pourcentage de reussites par sentiment : [0.8394557823129252, 0.6338709677419355, 0.7145877378435518]
Matrice des resultats : 
[[617. 153.  71.]
 [ 87. 393.  64.]
 [ 31.  74. 338.]]


# **Test Manuel de Phrases :**

In [ ]:
# Donne les probas données par le modele 'model' sur la phrase 'sentence'
# Exemple : proba = evaluate_proba(model,'The flight is good')

def evaluate_proba(model,sentence):
  sentence_preprocessed = text_preprocessing(sentence)
  #print(sentence_preprocessed)
  sentence_tokenized = tokenize_reviews(sentence_preprocessed)
  sentence_padded,p =PaddingX([sentence_tokenized],padd)
  return model.predict(sentence_padded)



# Donne le sentiment donné par le modele 'model' sur la phrase 'sentence'   (ATTENTION A COMMENT SONT NUMEROTES LES SENTIMENTS)
# Exemple : sentiment = evaluate_sentiment(model,'The flight is good')
def evaluate_sentiment(model,sentence):
  return np.argmax(evaluate_proba(model,sentence))

Ici vous pouvez tester le modèle sur des exemples :

In [ ]:
# Exemple :  proba = evaluate_proba(model,'The flight is good')
#            sentiment = evaluate_sentiment(model,'The flight is good')

#sentence = "this flight is good"
sentence = "this flight is not the best"
#sentence = "I love this movie"
#sentence = "I don't love this movie"
#sentence = "I wish this movie was better"

proba = evaluate_proba(model_trained,sentence)
print("Les probas sont : "+str(proba))
sentiment= evaluate_sentiment(model_trained,sentence)
print("Le sentiment est : "+str(sentiment))

Les probas sont : [[0.0759319  0.7354456  0.18862253]]
Le sentiment est : 1


# Sauvegarde du Modele

Sauvegarde le modele sur le drive du proprietaire :

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls '/content/gdrive/My Drive'

In [ ]:
model_trained.save('/content/gdrive/My Drive/model_Tweets_Flights')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/model_Tweets_Flights_pad70/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/model_Tweets_Flights_pad70/assets
